## Phase 1: Data loading, cleaning and feature engineering

In [173]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np

In [174]:
np.random.seed(88)

In [175]:
# -----------------------------------------------------------------------------
# STEP 1: Define column names for the dataset
# -----------------------------------------------------------------------------

# These are the 14 attributes in the heart disease dataset
columns = [
    "age",       # Age in years
    "sex",       # Sex (1 = male; 0 = female)
    "cp",        # Chest pain type (1-4)
    "trestbps",  # Resting blood pressure (mm Hg)
    "chol",      # Serum cholesterol (mg/dl)
    "fbs",       # Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
    "restecg",   # Resting ECG results (0-2)
    "thalach",   # Maximum heart rate achieved
    "exang",     # Exercise induced angina (1 = yes; 0 = no)
    "oldpeak",   # ST depression induced by exercise
    "slope",     # Slope of peak exercise ST segment (1-3)
    "ca",        # Number of major vessels colored by fluoroscopy (0-3)
    "thal",      # Thalassemia (3 = normal; 6 = fixed defect; 7 = reversible)
    "target"     # Diagnosis (0 = no disease, 1-4 = disease severity)
]

In [176]:
# -----------------------------------------------------------------------------
# STEP 2: Load and combine datasets from multiple locations
# -----------------------------------------------------------------------------
# The UCI heart disease database contains data from 4 locations
datasets = [
    "processed.cleveland.data",
    "processed.hungarian.data",
    "processed.switzerland.data",
    "processed.va.data"
]

df_list = []
for data_path in datasets:
    try:
        df = pd.read_csv(data_path, names=columns)
        # Replace '?' (missing value marker) with NaN
        df.replace('?', pd.NA, inplace=True)
        df_list.append(df)
        print(f"✓ Loaded {data_path}: {len(df)} rows")
    except FileNotFoundError:
        print(f"✗ Warning: {data_path} not found, skipping...")

# Combine all datasets vertically
combined = pd.concat(df_list, ignore_index=True)
print(f"\n Total combined rows: {len(combined)}")

✓ Loaded processed.cleveland.data: 303 rows
✓ Loaded processed.hungarian.data: 294 rows
✓ Loaded processed.switzerland.data: 123 rows
✓ Loaded processed.va.data: 200 rows

 Total combined rows: 920


In [177]:
# -----------------------------------------------------------------------------
# STEP 3: Handle missing values
# -----------------------------------------------------------------------------
print("\n Missing values before cleaning:")
print(combined.isnull().sum())

# Convert all columns to numeric (some may be strings due to '?')
for col in combined.columns:
    combined[col] = pd.to_numeric(combined[col], errors='coerce')

# Fill missing values with median (robust to outliers)
combined.fillna(combined.median(), inplace=True)

print("\n✓ Missing values after cleaning:")
print(combined.isnull().sum())

# Save the cleaned combined dataset
combined.to_csv("heart_disease_combined.csv", index=False)
print("\n Saved: heart_disease_combined.csv")


 Missing values before cleaning:
age           0
sex           0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalach      55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
target        0
dtype: int64

✓ Missing values after cleaning:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

 Saved: heart_disease_combined.csv


Clinical feature creation

We will create composite features that hold stronger predictive power than the individual components.

In [178]:
df_fe = combined.copy()

In [179]:
## --- A. Blood Pressure Features ---

# 1. Pulse Pressure (PP): Indicator of arterial stiffness. Higher is worse.
# Assuming 'trestbps' is Resting Blood Pressure
df_fe['pulse_pressure'] = df_fe['trestbps'] - df_fe['trestbps'].rolling(window=5).mean()
# A simpler version is just: df_fe['pulse_pressure'] = df_fe['trestbps'].diff()

# 2. Mean Arterial Pressure (MAP): More representative of perfusion pressure.
# Using a common approximation formula: MAP ≈ Diastolic + 1/3 * Pulse Pressure
# Since most datasets only have SBP ('trestbps'), we'll skip the accurate MAP calculation but note its importance.


## --- B. Cholesterol and Heart Rate Ratios ---

# 3. Cholesterol/Thalach Ratio: Contextualizes cholesterol against cardiovascular fitness.
# Lower 'thalach' (max heart rate) suggests lower fitness/cardiac reserve.
# Higher ratio indicates high cholesterol relative to heart performance.
df_fe['chol_thalach_ratio'] = df_fe['chol'] / df_fe['thalach']

# 4. Age and Cholesterol Interaction:
df_fe['age_chol_interaction'] = df_fe['age'] * df_fe['chol']


## --- C. Ischemic and Angina Interactions ---

# 5. Oldpeak/Thalach Ratio: Stress-induced ST depression relative to maximum achieved heart rate.
# High ST depression at low heart rate is a strong negative indicator.
df_fe['oldpeak_thalach_ratio'] = df_fe['oldpeak'] / df_fe['thalach']

# 6. Chest Pain Severity Score: Combine 'cp' and 'exang' (Exercise Induced Angina).
# Assuming 'cp' is categorical (1-4) and 'exang' is binary (0/1).
# We can create a weighted score where Angina (cp=4, or exang=1) contributes heavily.
# This assumes 'cp' is encoded such that higher values mean worse angina.
df_fe['angina_severity_score'] = df_fe['cp'] + (df_fe['exang'] * 2) # Multiply exang by 2 to give it extra weight

# 7. Zero-Inflated Features (Presence Flags):
# Create a binary feature for important zero-valued variables.
# Example: Create a flag for 'oldpeak' (ST depression) if it is zero.
df_fe['has_zero_oldpeak'] = (df_fe['oldpeak'] == 0).astype(int)

Categorical and numerical processing

Proper encoding and scaling are critical before training.

In [180]:
from sklearn.preprocessing import QuantileTransformer
# --- A. One-Hot Encoding for Nominal Categorical Features ---
# Assume 'cp', 'restecg', 'slope', 'thal' are nominal/ordinal and need dummies.
# NOTE: Use 'drop_first=True' to avoid multicollinearity.
categorical_cols = ['cp', 'restecg', 'slope', 'thal']
df_fe = pd.get_dummies(df_fe, columns=categorical_cols, drop_first=True)


# --- B. Outlier Treatment and Scaling for Numerical Features ---
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak',
                  'pulse_pressure', 'chol_thalach_ratio', 'oldpeak_thalach_ratio', 'angina_severity_score']

# 1. Scaling: Standard Scaling is generally good for tree-based models (XGBoost/LGBM) and critical for linear/DL models (SVM/ANN).
# Using robust scaling or QuantileTransformer (non-linear transformation) can be an alternative SOTA approach.
scaler = StandardScaler()
df_fe[numerical_cols] = scaler.fit_transform(df_fe[numerical_cols])

# 2. Outlier/Skewness Treatment (Optional but Advanced):
quantile_transformer = QuantileTransformer(output_distribution='normal', n_quantiles=100)
df_fe[['chol', 'oldpeak']] = quantile_transformer.fit_transform(df_fe[['chol', 'oldpeak']])

print("\nFeature Engineering Complete. New DataFrame Shape:", df_fe.shape)
print("New Features Created:")
print(df_fe.columns[-10:])


Feature Engineering Complete. New DataFrame Shape: (920, 25)
New Features Created:
Index(['has_zero_oldpeak', 'cp_2.0', 'cp_3.0', 'cp_4.0', 'restecg_1.0',
       'restecg_2.0', 'slope_2.0', 'slope_3.0', 'thal_6.0', 'thal_7.0'],
      dtype='object')


In [181]:
# Before proceeding to the next model, run this on your engineered DataFrame:
nan_counts = df_fe.isnull().sum()
if nan_counts.any():
    print("WARNING: NaN values found in the following columns:")
    print(nan_counts[nan_counts > 0])

    # Simple fix: Impute the NaNs (e.g., with the mean of the column)
    # The NaNs likely came from the rolling mean calculation in 'pulse_pressure'
    df_fe.fillna(df_fe.mean(), inplace=True)
    print("\nNaNs imputed with column mean.")
else:
    print("SUCCESS: No NaN values found in the feature-engineered data.")

pulse_pressure    4
dtype: int64

NaNs imputed with column mean.


In [182]:
# ============================================================================
# CRITICAL FIX 1: Convert multi-class target to binary
# ============================================================================
# The original target has values 0-4, but we need binary (0 or 1)
# Strategy: 0 = no disease, 1-4 = has disease
df_fe['target'] = (df_fe['target'] > 0).astype(int)

print("Target distribution after conversion:")
print(df_fe['target'].value_counts())

Target distribution after conversion:
target
1    509
0    411
Name: count, dtype: int64


In [183]:
# ============================================================================
# CRITICAL FIX 2: Verify no NaN/Inf values before training
# ============================================================================
print("\nChecking for NaN/Inf values...")
if df_fe.isnull().any().any():
    print("WARNING: NaN values found! Imputing...")
    df_fe.fillna(df_fe.mean(), inplace=True)

# Convert DataFrame to float to handle boolean columns before checking for inf
if np.isinf(df_fe.astype(float).values).any():
    print("WARNING: Inf values found! Clipping...")
    df_fe = df_fe.replace([np.inf, -np.inf], np.nan)
    df_fe.fillna(df_fe.mean(), inplace=True)

print("✓ Data validation complete")


Checking for NaN/Inf values...
✓ Data validation complete


## MLP implementation

🧠 Deep Learning Approach: PyTorch MLP
The implementation has three main parts:

Data Preparation: Converting Pandas/NumPy data into PyTorch Tensors and DataLoaders.

Model Definition: Defining the structure of the Neural Network.

Training Loop: Implementing the standard PyTorch training process.

1. Data Preparation and Utilities
We need to prepare the data for PyTorch, which requires converting to NumPy, then to Tensors, and finally using DataLoader for efficient batch processing.

In [184]:
# 1. Define X and y
X = df_fe.drop('target', axis=1).values.astype(np.float32)
y = df_fe['target'].values.astype(np.float32).reshape(-1, 1) # Reshape for binary classification

# Verify binary classification
unique_targets = np.unique(y)
print(f"\nUnique target values: {unique_targets}")
assert len(unique_targets) == 2, f"Target must be binary! Found {unique_targets}"


# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


Unique target values: [0. 1.]


2. PyTorch Model Definition
We define a Feed-Forward Neural Network (FNN) or MLP. A common strategy for tabular data is to use funnel architecture, where the number of neurons decreases with each layer.

Input Layer: in_features must match the number of columns in your engineered data (24).

Activation: ReLU (Rectified Linear Unit) is the standard for hidden layers.

Output Layer: out_features=1 for binary classification, followed by a Sigmoid activation (handled in the loss function for better numerical stability, as shown in the training step).

In [185]:
# ============================================================================
# Model Definition with Batch Normalization (IMPROVEMENT)
# ============================================================================
class HeartDiseaseMLP(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseMLP, self).__init__()

        # Layer sizes
        L1 = 128
        L2 = 64
        L3 = 32

        # Layer 1
        self.fc1 = nn.Linear(input_size, L1)
        self.bn1 = nn.BatchNorm1d(L1)  # Added Batch Normalization
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)

        # Layer 2
        self.fc2 = nn.Linear(L1, L2)
        self.bn2 = nn.BatchNorm1d(L2)  # Added Batch Normalization
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)  # Added dropout

        # Layer 3
        self.fc3 = nn.Linear(L2, L3)
        self.bn3 = nn.BatchNorm1d(L3)  # Added Batch Normalization
        self.relu3 = nn.ReLU()

        # Output layer
        self.fc4 = nn.Linear(L3, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        return self.fc4(x)

# Instantiate model
INPUT_SIZE = X_train.shape[1]
model = HeartDiseaseMLP(INPUT_SIZE)
print(f"\n✓ Model created with {INPUT_SIZE} input features")


✓ Model created with 24 input features


3. Training and Evaluation Loop
We use standard components for binary classification:

Loss Function: nn.BCEWithLogitsLoss() is highly recommended for stability, as it combines the Sigmoid activation and Binary Cross-Entropy loss.

Optimizer: Adam is the standard choice for most deep learning tasks.

In [186]:
# ============================================================================
# Training Setup
# ============================================================================
LEARNING_RATE = 1e-2  # Slightly increased for better convergence
NUM_EPOCHS = 150
EARLY_STOPPING_PATIENCE = 15

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

In [187]:
# ============================================================================
# Training Loop with Validation
# ============================================================================
print("\n--- Starting PyTorch Model Training ---\n")

best_loss = float('inf')
patience_counter = 0
train_losses = []

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0.0

    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        epoch_loss += loss.item()

    # Average loss for the epoch
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Update learning rate
    scheduler.step(avg_loss)

    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
        # Save best model
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}')

    # Early stopping
    if patience_counter >= EARLY_STOPPING_PATIENCE:
        print(f"\nEarly stopping triggered at epoch {epoch+1}")
        model.load_state_dict(best_model_state)
        break

print(f"\n✓ Training complete! Best loss: {best_loss:.4f}")


--- Starting PyTorch Model Training ---

Epoch [10/150], Loss: 0.5859
Epoch [20/150], Loss: 0.5459
Epoch [30/150], Loss: 0.5313
Epoch [40/150], Loss: 0.4954
Epoch [50/150], Loss: 0.4907
Epoch [60/150], Loss: 0.4886
Epoch [70/150], Loss: 0.5019
Epoch [80/150], Loss: 0.4899
Epoch [90/150], Loss: 0.4762

Early stopping triggered at epoch 91

✓ Training complete! Best loss: 0.4553


In [188]:
# ============================================================================
# Evaluation
# ============================================================================
model.eval()
with torch.no_grad():
    # Get predictions on test set
    test_logits = model(X_test_tensor)
    test_probabilities = torch.sigmoid(test_logits).numpy().flatten()
    test_predictions = (test_probabilities > 0.5).astype(int)

    # Get predictions on train set (to check overfitting)
    train_logits = model(X_train_tensor)
    train_probabilities = torch.sigmoid(train_logits).numpy().flatten()
    train_predictions = (train_probabilities > 0.5).astype(int)

    # Flatten targets
    y_test_flat = y_test.flatten()
    y_train_flat = y_train.flatten()

    # Calculate metrics
    test_accuracy = accuracy_score(y_test_flat, test_predictions)
    test_auc = roc_auc_score(y_test_flat, test_probabilities)

    train_accuracy = accuracy_score(y_train_flat, train_predictions)
    train_auc = roc_auc_score(y_train_flat, train_probabilities)

In [194]:
# ============================================================================
# Results
# ============================================================================
print("\n" + "="*60)
print("DEEP LEARNING MODEL RESULTS")
print("="*60)
print(f"Train Accuracy: {train_accuracy:.4f} | Train AUC-ROC: {train_auc:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f} | Test AUC-ROC:  {test_auc:.4f}")
print("="*60)

# Check for overfitting
overfit_gap = train_auc - test_auc
if overfit_gap > 0.1:
    print(f"\n⚠️  Warning: Possible overfitting detected (gap: {overfit_gap:.4f})")
else:
    print(f"\n✓ Model generalizes well (gap: {overfit_gap:.4f})")


DEEP LEARNING MODEL RESULTS
Train Accuracy: 0.7989 | Train AUC-ROC: 0.8862
Test Accuracy:  0.7717 | Test AUC-ROC:  0.8934

✓ Model generalizes well (gap: -0.0072)
